In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

from tensorflow.python.keras.utils import tf_utils

import sys
sys.path.append('../')

from layers.noisy_bn import NoisyBatchNormalization
%load_ext autoreload
%autoreload 2

In [88]:
class TestLayer(keras.layers.Layer):
        
    def call(self, inputs, training=None):
        if training is None:
            training = keras.backend.learning_phase()
        
        train_fn = lambda: tf.zeros(tf.shape(inputs))
        eval_fn = lambda: tf.ones(tf.shape(inputs))
            
        return tf_utils.smart_cond(training, train_fn, eval_fn)

In [67]:
keras.backend.learning_phase()

0

In [89]:
class MyModel(keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer = TestLayer()
        
    def call(self, inputs, training=None):
        return self.layer(inputs, training)

In [90]:
model = MyModel()

In [91]:
X = np.random.rand(97,100)
y = np.ones((97,100))

In [94]:
model(X, False)

<tf.Tensor: id=878, shape=(97, 100), dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>

In [95]:
model.compile(optimizer='adam',loss='mse')

In [96]:
model.fit(x=X, y=y, epochs=5)

Epoch 1/5
97/97 [==============================] - 0s 740us/sample - loss: 1.0000
Epoch 2/5
97/97 [==============================] - 0s 72us/sample - loss: 1.0000
Epoch 3/5
97/97 [==============================] - 0s 113us/sample - loss: 1.0000
Epoch 4/5
97/97 [==============================] - 0s 72us/sample - loss: 1.0000
Epoch 5/5
97/97 [==============================] - 0s 82us/sample - loss: 1.0000


In [97]:
model.predict(X)

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [9]:
a = np.array([[1.,2.,3.,4.,5.], [5.,4.,3.,2.,1.]], dtype=np.float32)
b = np.random.rand(2,2,2,2).astype(np.float32)
norm1 = keras.layers.BatchNormalization()
norm2 = keras.layers.BatchNormalization()

In [15]:
for x in range(1000):
    _ = norm1(a, training=True)
    _ = norm2(a, training=True)

In [16]:
norm1(a, training=False), norm2(a, training=False)

(<tf.Tensor: id=54513, shape=(2, 5), dtype=float32, numpy=
 array([[-0.9998265 , -0.99937093,  0.00400543,  0.99962974,  0.9999559 ],
        [ 0.9999559 ,  0.99962974,  0.00400543, -0.99937093, -0.9998265 ]],
       dtype=float32)>,
 <tf.Tensor: id=54526, shape=(2, 5), dtype=float32, numpy=
 array([[-0.9998265 , -0.99937093,  0.00400543,  0.99962974,  0.9999559 ],
        [ 0.9999559 ,  0.99962974,  0.00400543, -0.99937093, -0.9998265 ]],
       dtype=float32)>)

In [10]:
noisy_batch = NoisyBatchNormalization(alpha=0.0)

In [ ]:
for _ in range(1000):
    noisy_batch(a, training=True)

In [19]:
noisy_batch(a, training=False)

tf.Tensor(
[[1. 2. 3. 4. 5.]
 [5. 4. 3. 2. 1.]], shape=(2, 5), dtype=float32) tf.Tensor([[2.9998705 2.9998705 2.9998705 2.9998705 2.9998705]], shape=(1, 5), dtype=float32) tf.Tensor([[3.9998705e+00 1.0000000e+00 4.3171676e-05 1.0000000e+00 3.9998705e+00]], shape=(1, 5), dtype=float32) 0.001


<tf.Tensor: id=102590, shape=(2, 5), dtype=float32, numpy=
array([[-0.99982643, -0.999371  ,  0.00400832,  0.9996298 ,  0.9999559 ],
       [ 0.9999559 ,  0.9996298 ,  0.00400832, -0.999371  , -0.99982643]],
      dtype=float32)>

In [11]:
norm1(b, training=True), norm2(b, training=True)

(<tf.Tensor: id=341, shape=(2, 2, 2, 2), dtype=float32, numpy=
 array([[[[-0.73704046, -0.18121241],
          [ 1.2484467 ,  0.9654737 ]],
 
         [[-0.8000138 ,  1.3377585 ],
          [ 1.0506831 ,  0.6130806 ]]],
 
 
        [[[-0.95196944, -1.6271454 ],
          [-1.1226288 ,  0.39589506]],
 
         [[-0.06086094, -1.3944387 ],
          [ 1.3733833 , -0.10941248]]]], dtype=float32)>,
 <tf.Tensor: id=398, shape=(2, 2, 2, 2), dtype=float32, numpy=
 array([[[[-0.73704046, -0.18121241],
          [ 1.2484467 ,  0.9654737 ]],
 
         [[-0.8000138 ,  1.3377585 ],
          [ 1.0506831 ,  0.6130806 ]]],
 
 
        [[[-0.95196944, -1.6271454 ],
          [-1.1226288 ,  0.39589506]],
 
         [[-0.06086094, -1.3944387 ],
          [ 1.3733833 , -0.10941248]]]], dtype=float32)>)

In [24]:
noisy_batch(b, training=True)

<tf.Tensor: id=886, shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[-0.73704046, -0.18121223],
         [ 1.2484468 ,  0.9654739 ]],

        [[-0.8000138 ,  1.3377587 ],
         [ 1.0506833 ,  0.6130808 ]]],


       [[[-0.95196944, -1.627145  ],
         [-1.1226288 ,  0.3958952 ]],

        [[-0.06086089, -1.3944385 ],
         [ 1.3733834 , -0.10941231]]]], dtype=float32)>

In [35]:
def batchnorm_forward(X):
    mu = np.mean(X, axis=(0,1,2))
    var = np.var(X, axis=(0,1,2))

    X_norm = (X - mu) / np.sqrt(var + 0.001)
    out = 1 * X_norm 

    return out

In [37]:
batchnorm_forward(b.astype(np.float64))

array([[[[-0.73704042, -0.18121228],
         [ 1.24844679,  0.9654738 ]],

        [[-0.80001376,  1.33775863],
         [ 1.05068317,  0.61308071]]],


       [[[-0.9519694 , -1.62714515],
         [-1.12262878,  0.39589516]],

        [[-0.06086089, -1.39443851],
         [ 1.3733833 , -0.10941236]]]])

In [11]:
norm1(b, training=False), norm2(b, training=False)

(<tf.Tensor: id=290, shape=(2, 2, 2, 2), dtype=float32, numpy=
 array([[[[0.34406313, 0.04382638],
          [0.40071642, 0.7218602 ]],
 
         [[0.11433753, 0.40438968],
          [0.8774112 , 0.38251224]]],
 
 
        [[[0.754469  , 0.10119688],
          [0.15986466, 0.4807517 ]],
 
         [[0.57328904, 0.40044674],
          [0.9360982 , 0.5159821 ]]]], dtype=float32)>,
 <tf.Tensor: id=301, shape=(2, 2, 2, 2), dtype=float32, numpy=
 array([[[[0.34406313, 0.04382638],
          [0.40071642, 0.7218602 ]],
 
         [[0.11433753, 0.40438968],
          [0.8774112 , 0.38251224]]],
 
 
        [[[0.754469  , 0.10119688],
          [0.15986466, 0.4807517 ]],
 
         [[0.57328904, 0.40044674],
          [0.9360982 , 0.5159821 ]]]], dtype=float32)>)

In [15]:
norm1.moving_mean, norm1.moving_variance

(<tf.Variable 'batch_normalization_v2_2/moving_mean:0' shape=(2,) dtype=float32, numpy=array([0.00523175, 0.00383578], dtype=float32)>,
 <tf.Variable 'batch_normalization_v2_2/moving_variance:0' shape=(2,) dtype=float32, numpy=array([0.9909894, 0.9904777], dtype=float32)>)

In [16]:
noisy_batch.moving_mean, noisy_batch.moving_variance

(<tf.Variable 'noisy_batch_normalization_1/moving_mean:0' shape=(2,) dtype=float32, numpy=array([0.00523176, 0.00383578], dtype=float32)>,
 <tf.Variable 'noisy_batch_normalization_1/moving_variance:0' shape=(2,) dtype=float32, numpy=array([0.9908657, 0.990418 ], dtype=float32)>)

In [12]:
noisy_batch(b, training=False)

[0, 1, 2]


<tf.Tensor: id=330, shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0.34408453, 0.04382769],
         [0.40074137, 0.72188187]],

        [[0.11434464, 0.4044018 ],
         [0.87746584, 0.38252372]]],


       [[[0.75451595, 0.1011999 ],
         [0.15987462, 0.48076612]],

        [[0.57332474, 0.40045875],
         [0.9361565 , 0.5159976 ]]]], dtype=float32)>

In [28]:
noisy_batch.gamma, noisy_batch.beta

(<tf.Variable 'noisy_batch_normalization_1/gamma:0' shape=(2,) dtype=float32, numpy=array([1., 1.], dtype=float32)>,
 <tf.Variable 'noisy_batch_normalization_1/beta:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>)

In [ ]:
for x in range(1000):
    _ = norm1(b, training=True)
    _ = norm2(b, training=True)

In [ ]:
for _ in range(1000):
    noisy_batch(b, training=True)

In [ ]:
noisy_batch(b, b)